In [2]:
# import stuff based on the provided assignments

import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
from scipy.stats import norm, lognorm
import pandas as pd
from pyfinance.options import BSM
from mpl_toolkits.mplot3d import Axes3D
from datar import f
from datar.dplyr import mutate, filter, if_else, pull, group_by, select
from pprint import pprint
import plotly.express as px

[2021-12-11 16:30:46][datar][WARNING] Builtin name "filter" has been overriden by datar.


In [3]:
"""function for testing frequencies and different strikes for all worksheets"""

def read_data(xls,worksheet):
    return pd.read_excel(xls, worksheet)
    

def format_data(df,expiry):
    """Convert into correct format"""
    df.rename(dict(zip(np.append(np.array(df.columns[0]), df.columns[-3:]), ['T', 'S', 'r', 'date'])), axis='columns', inplace=True)
    df = df.melt(id_vars=['T', 'r', 'S', 'date'], var_name="E").dropna()
    df['date'] = pd.to_datetime(df['date'], dayfirst=True)
    df['name'] = df['E'].astype('str') + '-' + expiry
    
    df.rename({'value':'Cobs'}, axis='columns', inplace=True)
    df['E'] = df['E'].astype('float64')
    df = df  >> mutate(r = f.r/100, Mat = f.T, T = f.T/252, S = f.S/1000, E = f.E/1000, Cobs = f.Cobs/1000)
    df['Mat'] = df['Mat'].astype('int')
    
    return df
    
def calculate_bs_greeks(row):
    """Calculate IV and using that as volatility find delta and vega"""
    S=row['S']
    E=row['E']
    r=row['r']
    T=row['T']
    Cobs=row['Cobs']
    
    starting_vol = .2
    bsm = BSM(S, E, T, r, starting_vol)
    iv = bsm.implied_vol(Cobs, precision=1.0e-6)
    
    bsm_iv = BSM(S, E, T, r, iv)
    
    delta = bsm_iv.delta()
    vega = bsm_iv.vega()
    gamma = bsm_iv.gamma()
    
    return iv, delta, vega, gamma

def hedge(p_o, p_s, cobs, S, delta):
    option_gain = cobs - p_o
    stock_gain = -delta * (S - p_s['price'])
    error = option_gain + stock_gain
    
    return cobs, {'price':S, 'delta':delta}, error**2

def calculate_sheet_greeks(excel, sheet_name, sheetnr):
    day = str(sheet_name[3:5])
    month = str(sheet_name[5:7])
    year = str(sheet_name[7:11])
    expiry=year+month+day

    
    df = format_data(read_data(xls, sheet_name), expiry)
    df = mutate(df, Cobs = if_else(f.Cobs > 1, f.Cobs/1000, f.Cobs))
    greeks = df.apply(calculate_bs_greeks, axis=1, result_type='expand')
    df[['iv', 'delta', 'vega', 'gamma']] = greeks
    df['sheet'] = sheet_name
    df = filter(df, 0 < f.iv)
    
    return df



In [4]:
# read data and calculate IV

xls = pd.ExcelFile('data/isx2010C.xls')
sheets = xls.sheet_names

sheet_order = [0,1,11,10,9,8,7,6,5,4,3,2]

all_sheets = []

for i in sheet_order:
    df = calculate_sheet_greeks(xls, sheets[i], i)
    all_sheets.append(df)

all_data = pd.concat(all_sheets)

/home/juuso/Documents/school/finaeng1/venv/lib/python3.9/site-packages/pyfinance/options.py:151: RuntimeWarning: overflow encountered in double_scalars
  + (self.r + 0.5 * self.sigma ** 2) * self.T
/home/juuso/Documents/school/finaeng1/venv/lib/python3.9/site-packages/pyfinance/options.py:242: RuntimeWarning: divide by zero encountered in double_scalars
  vol = vol + diff / opt.vega()
/home/juuso/Documents/school/finaeng1/venv/lib/python3.9/site-packages/pyfinance/options.py:149: RuntimeWarning: invalid value encountered in double_scalars
  self.d1 = (
/home/juuso/Documents/school/finaeng1/venv/lib/python3.9/site-packages/pyfinance/options.py:242: RuntimeWarning: overflow encountered in double_scalars
  vol = vol + diff / opt.vega()


In [5]:
all_data

,T,r,S,date,E,Cobs,name,Mat,iv,delta,vega,gamma,sheet
,<float64>,<float64>,<float64>,<datetime64[ns]>,<float64>,<float64>,<object>,<int64>,<float64>,<float64>,<float64>,<float64>,<object>
0,0.341270,0.0011,0.49134,2009-09-21,0.34,0.15220,340-20100115,86,0.322557,0.979826,0.014000,0.526812,isx15012010C
1,0.337302,0.0011,0.49435,2009-09-22,0.34,0.15505,340-20100115,85,0.316441,0.983437,0.011838,0.453820,isx15012010C
2,0.333333,0.0011,0.49036,2009-09-23,0.34,0.15240,340-20100115,84,0.386955,0.960153,0.024321,0.784158,isx15012010C
4,0.325397,0.0011,0.48411,2009-09-25,0.34,0.14555,340-20100115,82,0.354207,0.967964,0.019839,0.734450,isx15012010C
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5,0.321429,0.0011,0.49169,2009-09-28,0.34,0.15280,340-20100115,81,0.350097,0.974979,0.016304,0.599283,isx15012010C
8005,0.019841,0.0014,0.55781,2010-12-13,0.56,0.00310,560-20101217,5,0.130409,0.419718,0.030709,38.143167,isx17122010C
8006,0.015873,0.0014,0.55864,2010-12-14,0.56,0.00245,560-20101217,4,0.109453,0.433362,0.027686,51.062978,isx17122010C
8007,0.011905,0.0014,0.55623,2010-12-15,0.56,0.00130,560-20101217,3,0.114676,0.297266,0.021014,49.750953,isx17122010C


In [37]:
# buying a butterfly spread and hedging it with stock
# butterfly spread = long call with strike E_1, short 2 call with same maturity strike E_2 where E_1 < E_2

# choose which strikes used to create spread
strikes = [0.35, 0.4, 0.45, 0.48, 0.5, 0.52,0.54]

freq_min = 1
freq_max = 6
observed_data = []
    
for call_1 in strikes:
    for call_2 in strikes:
        for call_3 in strikes:
            
            if call_1 < call_2 < call_3:
                print(call_1,call_2,call_3)
                long_call_1 = call_1
                short_call = call_2
                long_call_2 = call_3
                long_call_data_1 = all_data[all_data['E'] == long_call_1]
                short_call_data = all_data[all_data['E'] == short_call]
                long_call_data_2 = all_data[all_data['E'] == long_call_2]
                for long_call_name in long_call_data_1['name'].unique():
                    unique_long_call_data = long_call_data_1[long_call_data_1['name']==long_call_name]
                    hedge_data = unique_long_call_data.merge(short_call_data, on=['date', 'Mat', 'S', 'T', 'r'])
                    hedge_data = hedge_data.merge(long_call_data_2, on=['date', 'Mat', 'S', 'T', 'r'])
                    if not hedge_data.empty:
                        for freq in range(freq_min, freq_max):
                            hedge_data['stock_amount'] = -(hedge_data.iloc[::freq,:]['delta_x']-2*hedge_data.iloc[::freq,:]['delta_y']+hedge_data.iloc[::freq,:]['delta'])
                            hedge_data['stock_amount']=hedge_data['stock_amount'].ffill()

                            hedge_data['option_return'] = hedge_data['Cobs_x'].diff()+hedge_data['Cobs'].diff() - 2*hedge_data['Cobs_y'].diff()
                            hedge_data['stock_return'] = hedge_data['stock_amount'].shift(1) * (hedge_data['S'].diff())
                            hedge_data['portfolio_return'] = hedge_data['stock_return'] + hedge_data['option_return']

                            s_start = hedge_data['S'].iloc[0]
                            s_end = hedge_data['S'].iloc[-1]
                            s_mean = hedge_data['S'].mean()
                            mse = (hedge_data['portfolio_return'] ** 2).mean()
                            sheet = unique_long_call_data['sheet'].iloc[0]

                            observed_data.append([long_call_1, long_call_2, short_call,s_start,s_end,s_mean,mse, freq, sheet])

0.35 0.4 0.45
0.35 0.4 0.48
0.35 0.4 0.5
0.35 0.4 0.52
0.35 0.4 0.54
0.35 0.45 0.48
0.35 0.45 0.5
0.35 0.45 0.52
0.35 0.45 0.54
0.35 0.48 0.5
0.35 0.48 0.52
0.35 0.48 0.54
0.35 0.5 0.52
0.35 0.5 0.54
0.35 0.52 0.54
0.4 0.45 0.48
0.4 0.45 0.5
0.4 0.45 0.52
0.4 0.45 0.54
0.4 0.48 0.5
0.4 0.48 0.52
0.4 0.48 0.54
0.4 0.5 0.52
0.4 0.5 0.54
0.4 0.52 0.54
0.45 0.48 0.5
0.45 0.48 0.52
0.45 0.48 0.54
0.45 0.5 0.52
0.45 0.5 0.54
0.45 0.52 0.54
0.48 0.5 0.52
0.48 0.5 0.54
0.48 0.52 0.54
0.5 0.52 0.54


In [38]:

obs_data = pd.DataFrame(observed_data, columns=['long_call_1','long_call_2','short_call','S_start', 'S_end', 'S_mean', 'mse', 'freq', 'sheet'])
obs_data['spread_name'] = obs_data['long_call_1'].astype('str')+obs_data['short_call'].astype('str')+obs_data['long_call_2'].astype('str')
obs_data['spread_size'] = obs_data['long_call_2']-obs_data['long_call_1']

In [39]:

fig = px.scatter(obs_data, x='S_mean', y='mse', color='freq')
fig.show()

In [44]:

fig = px.scatter(obs_data, x='spread_size', y='mse', color='freq')
fig.show()

In [42]:
import statsmodels.api as sm

reg = obs_data.copy()

freq_dummy = pd.get_dummies(reg['freq'],prefix='freq')
sheet_dummy = pd.get_dummies(reg['sheet'],prefix='sheet')

reg_df = pd.concat([reg, freq_dummy, sheet_dummy], axis=1)
X = reg_df[['S_mean'] + list(freq_dummy.columns)[:-1] + list(sheet_dummy.columns)[:-1]]
y = reg_df['mse']

X = sm.add_constant(X)
model = sm.OLS(y,X)
results = model.fit()

print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                    mse   R-squared:                       0.205
Model:                            OLS   Adj. R-squared:                  0.199
Method:                 Least Squares   F-statistic:                     31.72
Date:                Sun, 12 Dec 2021   Prob (F-statistic):           2.57e-86
Time:                        11:31:52   Log-Likelihood:                 18403.
No. Observations:                1985   AIC:                        -3.677e+04
Df Residuals:                    1968   BIC:                        -3.668e+04
Df Model:                          16                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
const                 -0.0001   3.46

In [43]:
X = reg_df[['S_mean','spread_size'] + list(freq_dummy.columns)[:-1] + list(sheet_dummy.columns)[:-1]]
y = reg_df['mse']

X = sm.add_constant(X)
model = sm.OLS(y,X)
results = model.fit()

print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                    mse   R-squared:                       0.206
Model:                            OLS   Adj. R-squared:                  0.200
Method:                 Least Squares   F-statistic:                     30.09
Date:                Sun, 12 Dec 2021   Prob (F-statistic):           2.78e-86
Time:                        11:32:15   Log-Likelihood:                 18404.
No. Observations:                1985   AIC:                        -3.677e+04
Df Residuals:                    1967   BIC:                        -3.667e+04
Df Model:                          17                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
const                 -0.0001    3.5